In [45]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay

def select_points(img, title="Select points"):
    plt.imshow(img, cmap='gray')
    plt.title(title)
    points = plt.ginput(n=-1, timeout=0)
    plt.close()
    return np.array(points)

def add_boundary_points(img_shape):
    h, w = img_shape[:2]
    return np.array([
        [0, 0], [w-1, 0], [w-1, h-1], [0, h-1],
        [w//2, 0], [w-1, h//2], [w//2, h-1], [0, h//2]
    ])


In [ ]:
# Load and resize images
img1 = cv2.imread('face_img.jpg', cv2.IMREAD_COLOR)
img2 = cv2.imread('lion_img.jpg', cv2.IMREAD_COLOR)
h, w = img1.shape[:2]
img2 = cv2.resize(img2, (w, h))  # Make sure they are same size!

In [ ]:
# Select points
points1 = select_points(img1, "Select keypoints for Woman")
points2 = select_points(img2, "Select corresponding keypoints for Lion")


In [ ]:
# Add boundary points
boundary = add_boundary_points(img1.shape)
points1 = np.vstack([points1, boundary])
points2 = np.vstack([points2, boundary])

In [ ]:
# Triangulate based on average shape
average_points = (points1 + points2) / 2
tri = Delaunay(average_points)

# Plot the triangulation
plt.triplot(average_points[:,0], average_points[:,1], tri.simplices)
plt.plot(average_points[:,0], average_points[:,1], 'o')
plt.title("Triangulation")
plt.show()

In [40]:
alpha = 0.5  # Half woman, half lion
morphed_img = np.zeros_like(img1, dtype=np.float32)


In [41]:
for tri_indices in tri.simplices:
    # Get vertices of current triangle
    x1 = points1[tri_indices]
    x2 = points2[tri_indices]
    x = average_points[tri_indices]  # Intermediate points

    # Compute affine transforms
    warp_mat1 = cv2.getAffineTransform(np.float32(x1), np.float32(x))
    warp_mat2 = cv2.getAffineTransform(np.float32(x2), np.float32(x))

    # Warp each triangle separately
    img1_warped = cv2.warpAffine(img1, warp_mat1, (w, h))
    img2_warped = cv2.warpAffine(img2, warp_mat2, (w, h))

    # Create mask for the triangle
    mask = np.zeros((h, w), dtype=np.float32)
    cv2.fillConvexPoly(mask, np.int32(x), 1.0, 16, 0)

    # Add to morphed image
    morphed_img += mask * ((1 - alpha) * img1_warped + alpha * img2_warped)


In [42]:
morphed_img = np.clip(morphed_img, 0, 255).astype(np.uint8)
plt.imshow(morphed_img, cmap='gray')
plt.title('Morphed Image')
plt.axis('off')
plt.show()

In [78]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay

def morph_triangle(img1, img2, img, t1, t2, t, alpha):
    # Compute affine transform matrices
    warp_mat1 = cv2.getAffineTransform(np.float32(t1), np.float32(t))
    warp_mat2 = cv2.getAffineTransform(np.float32(t2), np.float32(t))

    # Warp triangles
    warped_img1 = cv2.warpAffine(img1, warp_mat1, (img.shape[1], img.shape[0]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101)
    warped_img2 = cv2.warpAffine(img2, warp_mat2, (img.shape[1], img.shape[0]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101)

    # Mask for the triangle
    mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.float32)
    cv2.fillConvexPoly(mask, np.int32(t), 1.0, 16, 0)

    # Blend the triangles
    img += mask * ((1.0 - alpha) * warped_img1 + alpha * warped_img2)

def morph_images(img1, img2, points1, points2, tri, alpha):
    h, w, c = img1.shape  
    morphed_img = np.zeros((h, w, c), dtype=np.float32)

    points = (1 - alpha) * points1 + alpha * points2

    for tri_indices in tri.simplices:
        x1 = points1[tri_indices]
        x2 = points2[tri_indices]
        x = points[tri_indices]

        # For each channel separately
        for ch in range(c):
            morph_triangle(img1[:,:,ch], img2[:,:,ch], morphed_img[:,:,ch], x1, x2, x, alpha)

    return np.clip(morphed_img, 0, 255).astype(np.uint8)


In [ ]:
# # Set up Video Writer
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4
# video_out = cv2.VideoWriter('morph_video.mp4', fourcc, 15.0, (w, h), isColor=True)  # 15 fps, grayscale video

# # Create morph frames
# num_frames = 100  # Number of frames in the video
# for i, alpha in enumerate(np.linspace(0, 1, num_frames)):
#     print(f"Generating frame {i+1}/{num_frames}...")
#     frame = morph_images(img1, img2, points1, points2, tri, alpha)
#     video_out.write(frame)

# video_out.release()
# print("Morphing video saved as 'morph_video.mp4'")

In [ ]:
# Set up Video Writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_out = cv2.VideoWriter('morph_loop.mp4', fourcc, 15.0, (w, h), isColor=True)  # 15 fps, color video

# Create morph frames
num_frames = 100  # Forward frames
frames = []

# Generate forward morph
for i, alpha in enumerate(np.linspace(0, 1, num_frames)):
    print(f"Generating forward frame {i+1}/{num_frames}...")
    frame = morph_images(img1, img2, points1, points2, tri, alpha)
    frames.append(frame)

# Now frames[] contains Woman → Lion morph

# Write forward frames
for frame in frames:
    video_out.write(frame)

# Write backward frames (skip last frame to avoid repeated frame)
for frame in frames[-2::-1]:  # Start second to last frame, reverse
    video_out.write(frame)

video_out.release()
print("video saved as 'morph_loop.mp4'")


Generating forward frame 1/100...
Generating forward frame 2/100...
Generating forward frame 3/100...
Generating forward frame 4/100...
Generating forward frame 5/100...
Generating forward frame 6/100...
Generating forward frame 7/100...
Generating forward frame 8/100...
Generating forward frame 9/100...
Generating forward frame 10/100...
Generating forward frame 11/100...
Generating forward frame 12/100...
Generating forward frame 13/100...
Generating forward frame 14/100...
Generating forward frame 15/100...
Generating forward frame 16/100...
Generating forward frame 17/100...
Generating forward frame 18/100...
Generating forward frame 19/100...
Generating forward frame 20/100...
Generating forward frame 21/100...
Generating forward frame 22/100...
Generating forward frame 23/100...
Generating forward frame 24/100...
Generating forward frame 25/100...
Generating forward frame 26/100...
Generating forward frame 27/100...
Generating forward frame 28/100...
Generating forward frame 29/1

In [76]:
import cv2

cap = cv2.VideoCapture('morph_loop.mp4')

if not cap.isOpened():
    print("Error opening video file")

print("Playing video... Press Enter in the terminal to exit.")

while True:
    ret, frame = cap.read()

    if not ret:
        # If video ended, restart from beginning
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        continue

    cv2.imshow('Morph Video Preview', frame)

    # Check for keypress every 30ms
    if cv2.waitKey(70) == 13:  # 13 is Enter key
        break

cap.release()
cv2.destroyAllWindows()


Playing video... Press Enter in the terminal to exit.
